# The Vanishing/Exploding Gradients Problems

Thuật toán lan truyền ngược (backpropagation) thực hiện bằng cách bắt đầu từ output đến input layer, lan truyền gradient lỗi trên đường đi. Khi tính toán đạo hàm hàm mất mát liên quan đến từng tham số trong mạng, nó dùng đạo hàm để cập nhật các tham số bằng cách giảm gradient.

Thật không may, độ dốc (gradient) thường ngày càng nhỏ dần khi thuật toán lan truyền ngược xuống các lớp thấp hơn. Kết quả là, việc cập nhật bằng phương pháp gradient descent gần như không làm thay đổi trọng số kết nối của các lớp thấp, và quá trình huấn luyện không bao giờ hội tụ đến một nghiệm tốt. Hiện tượng này được gọi là vấn đề gradient biến mất (vanishing gradients).

Trong một số trường hợp, điều ngược lại có thể xảy ra: gradient có thể ngày càng lớn hơn cho đến khi các lớp nhận được những cập nhật trọng số cực kỳ lớn và thuật toán bị phân kỳ. Đây được gọi là vấn đề gradient bùng nổ (exploding gradients), thường xuất hiện nhất trong các mạng nơ-ron hồi tiếp (recurrent neural networks – xem thêm ở Chương 15).

Nói chung, các mạng nơ-ron sâu gặp phải vấn đề gradient không ổn định (unstable gradients); các lớp khác nhau có thể học với tốc độ rất khác nhau.